<a href="https://colab.research.google.com/github/sasiru11/Simple-RAG-Application-With-LangChain/blob/main/SimpleRAGAplicationWithLangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simple RAG Aplication using LangChain and OpenAI

In [2]:
#Install the necessory packages

!pip install langchain -qU
!pip install langchain-openai -qU
!pip install langchain-chroma -qU

In [3]:
#import necessary libraries

import os
from google.colab import userdata

Initialize OpenAI LLM


In [4]:
from langchain_openai import ChatOpenAI

#Set OpenAI API Key

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

#Initialize the chat openai model

llm = ChatOpenAI(model = "gpt-3.5-turbo",temperature=0)

Initialize Embedding Model

In [5]:
from langchain_openai import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(model = "text-embedding-3-small")


Create an embeded Document

In [7]:
from langchain_core.documents import Document

#Define a list of documents with content and method

documents = [Document(page_content = "The T20 world cup 2024 is in full swing, bring excitement and drama to cricket fans world wide",metadata = {"source":"cricket news"}),
             Document(page_content = "The fifa world cup held that date",metadata = {"source":"Sport news"}),
             Document(page_content = "In 2025 AI will come forth",metadata = {"source":"AI news"})]

Save Document in vector Database

In [8]:
#Create a vector store using the documents and embedding models
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents,embedding = embedding_model,)

Similarity Search

In [19]:
result = vectorstore.similarity_search("test match")
for result in result:
  print("-----------------")
  print(result.page_content)
  print(result.metadata)


-----------------
The T20 world cup 2024 is in full swing, bring excitement and drama to cricket fans world wide
{'source': 'cricket news'}
-----------------
The fifa world cup held that date
{'source': 'Sport news'}
-----------------
In 2025 AI will come forth
{'source': 'AI news'}


Embed Query and perform similerity search by vector

In [21]:
#embed a query using the embedding model

query_embedding = embedding_model.embed_query("machine learning")

query_embedding[:10]

#peint the length of the query embedding
len(query_embedding)

1536

In [22]:
results = vectorstore.similarity_search_by_vector(query_embedding,)
for result in results:
  print("-----------------")
  print(result.page_content)
  print(result.metadata)

-----------------
In 2025 AI will come forth
{'source': 'AI news'}
-----------------
The fifa world cup held that date
{'source': 'Sport news'}
-----------------
The T20 world cup 2024 is in full swing, bring excitement and drama to cricket fans world wide
{'source': 'cricket news'}


Create Retriver

In [23]:
#create retriver from the vector

retriver = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":1}
)

In [24]:
#perform batch retrival using the retrival

batch_result = retriver.batch(["machine learning","test match"])

for result in batch_result:
  print("-----------------")
  for doc in result:
    print(doc.page_content)
    print(doc.metadata)

-----------------
In 2025 AI will come forth
{'source': 'AI news'}
-----------------
The T20 world cup 2024 is in full swing, bring excitement and drama to cricket fans world wide
{'source': 'cricket news'}


Create prompt template

In [25]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

#Define a message template for the Chatbot

message = """
              Answer the question using the provide context only
              {question}
              Contect{context}
              """

In [26]:
#Create a chatprompt template for the message

prompt = ChatPromptTemplate.from_messages([("human",message)])

Chain retriever and Prompt template with LLM

In [31]:
chain = {"context":retriver,"question":RunnablePassthrough()} | prompt | llm

response = chain.invoke("tell me something")
print(response.content)

In 2025, AI will come forth.
